# Toronto Neighbourhoods
### This notebook will be used to explore, segment, and cluster the neighborhoods in the city of Toronto

### 1. Creating the Dataframe
------------------------------------------------------------------------
<b>Importing libraries and scraping wikipedia for postalcode table</b>

In [2]:
import requests
import pandas as pd
import numpy as np

website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')

scrape_table = soup.find('table',{'class':'wikitable sortable'})
# scrape_table

<b>Converting scraped table to dataframe and ignoring cells with a borough that is Not assigned</b>

In [3]:
df_scraped = pd.read_html(str(scrape_table))
df_scraped = df_scraped[0].dropna(axis=0)
df_scraped = df_scraped[~df_scraped['Borough'].isin(['Not assigned'])]

- <b>Combining rows with common postal code and separating by comma</b>
<br>
- <b>If neighbourhood is not assigned, using Borough value as neighbourhood</b>

In [4]:
df_comma = df_scraped.groupby(['Postcode','Borough'], sort=False)['Neighbourhood'].apply(', '.join).reset_index()
# df_comma
df_comma['Neighbourhood'] = np.where(df_comma.Neighbourhood == 'Not assigned', df_comma.Borough, df_comma.Neighbourhood)
df_comma.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [5]:
df_comma.shape

(103, 3)

### 2. Adding Geographic coordinates
------------------------------------------------------------------------
<b>Using CSV to get Lat/Long intel</b>

In [6]:
df_coord = pd.read_csv('Geospatial_Coordinates.csv')
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
df_comma_coord = df_comma.merge(df_coord, how='inner', left_on='Postcode', right_on='Postal Code')
df_comma_coord.drop(['Postal Code'], axis=1, inplace=True)
df_comma_coord.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


### 3. Generating maps and visualizing neighborhoods
------------------------------------------------------------------------
<b>Using the Boroughs that contain Toronto</b>

In [8]:
toronto_data = df_comma_coord[df_comma_coord['Borough'].str.contains('Toronto')].reset_index(drop=True)

In [10]:
import folium

map_Toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto 

<b>Define Foursquare Credentials and Version</b>

In [11]:
CLIENT_ID = 'YG1TKWPHECOQQWLO3ICSIAXBQLVX24ZNVC1XRADRCXCNQFLC' # your Foursquare ID
CLIENT_SECRET = 'V0MK3UMF1MN4N3F4W1O1BDVDMAMU0LC5T4QVNANSXS5T5XUY' # your Foursquare Secret
VERSION = '20190718' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YG1TKWPHECOQQWLO3ICSIAXBQLVX24ZNVC1XRADRCXCNQFLC
CLIENT_SECRET:V0MK3UMF1MN4N3F4W1O1BDVDMAMU0LC5T4QVNANSXS5T5XUY


In [13]:
toronto_data.loc[0, 'Neighbourhood']

'Harbourfront, Regent Park'

In [15]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront, Regent Park are 43.6542599, -79.3606359.


- <b>Getting the top 100 venues that are Harbourfront/Regent Park within a radius of 500 meters.</b>
- <b>Creating the GET request URL and getting a response</b>

In [17]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL
results = requests.get(url).json()

In [18]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

<b>JSON cleanup and converting it into pandas dataframe.</b>

In [20]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Toronto Cooper Koo Family Cherry St YMCA Centre,Gym / Fitness Center,43.653191,-79.357947
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149


In [21]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

50 venues were returned by Foursquare.


------------------------------
### Exploring Neighborhoods in Toronto
------------------------------

In [23]:
#creating a function for api calling
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<b>Getting nearby venues and creating dataframe</b>

In [25]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'], latitudes=toronto_data['Latitude'], longitudes=toronto_data['Longitude'])

Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
Fir

In [27]:
#Resulting dataframe and its size
print(toronto_venues.shape)
toronto_venues.head()

(1702, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront, Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Harbourfront, Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


Counting venues returned for each neighbourhood

In [29]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"Brockton, Exhibition Place, Parkdale Village",23,23,23,23,23,23
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",15,15,15,15,15,15
"Cabbagetown, St. James Town",45,45,45,45,45,45
Central Bay Street,83,83,83,83,83,83
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16


In [ ]:
<b>Getting nearby venues and creating dataframe</b>

In [30]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
toronto_onehot.shape

(1702, 236)

<b>Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category</b>

In [36]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.010000,0.000000,0.000000,0.010000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.018182,0.000000,0.000000,0.000000,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.000000,0.066667,0.066667,0.066667,0.133333,0.133333,0.133333,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022222,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.012048,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012048,...,0.000000,0.00000,0.000000,0.00000,0.00,0.012048,0.000000,0.000000,0.012048,0.000000
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.01000,0.000000,0.00000,0.00,0.060000,0.000000,0.040000,0.010000,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.011765,0.011765,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011765,...,0.011765,0.00000,0.000000,0.00000,0.00,0.000000,0.011765,0.011765,0.000000,0.011765


In [37]:
toronto_grouped.shape

(38, 236)

<b>Printing each neighbourhood's top 5 most common venues</b>

In [39]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2  Thai Restaurant  0.04
3       Steakhouse  0.04
4              Bar  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.11
1        Cocktail Bar  0.05
2            Beer Bar  0.04
3          Steakhouse  0.04
4  Seafood Restaurant  0.04


----Brockton, Exhibition Place, Parkdale Village----
                venue  freq
0      Breakfast Spot  0.09
1                Café  0.09
2         Coffee Shop  0.09
3         Yoga Studio  0.04
4  Italian Restaurant  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.12
1          Comic Shop  0.06
2       Auto Workshop  0.06
3                Park  0.06
4          Restaurant  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0    Airport Lounge  0.13
1   Air

<b>Creating a new dataframe to display the top 10 venues for each neighborhood.</b>

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,Bar,Cosmetics Shop,Asian Restaurant,Burger Joint,American Restaurant,Gym
1,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Bakery,Beer Bar,Steakhouse,Cheese Shop,Seafood Restaurant,Café,Jazz Club
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Breakfast Spot,Café,Yoga Studio,Stadium,Burrito Place,Restaurant,Caribbean Restaurant,Climbing Gym,Performing Arts Venue
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Smoke Shop,Brewery,Burrito Place,Restaurant,Skate Park,Auto Workshop,Fast Food Restaurant,Farmers Market,Spa
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Terminal,Airport Lounge,Boat or Ferry,Coffee Shop,Sculpture Garden,Boutique,Bar,Airport Gate,Airport Food Court


------------------------------
### Running k-means to cluster the neighborhood into 2 clusters.
------------------------------

In [86]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 15

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10])
print(len(kmeans.labels_))

[ 1  1  1  8 10  1  1  1  7  1]
38


In [87]:
# add clustering labels
neighborhoods_venues_sorted.drop(['Cluster Labels'], axis=1, inplace=True)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,1,Coffee Shop,Park,Café,Bakery,Theater,Mexican Restaurant,Pub,Restaurant,Breakfast Spot,Gym / Fitness Center
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Café,Fast Food Restaurant,Diner,Restaurant,Ramen Restaurant,Bookstore
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Hotel,Restaurant,Café,Italian Restaurant,Beer Bar,Breakfast Spot,Cocktail Bar,Cosmetics Shop,Gastropub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,12,Pub,Health Food Store,Trail,Wings Joint,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Cocktail Bar,Farmers Market,Bakery,Beer Bar,Steakhouse,Cheese Shop,Seafood Restaurant,Café,Jazz Club


<b>Visualizing clusters</b>

In [88]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

<b>Examining first 3 clusters</b>

In [89]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,East Toronto,0,Café,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Gastropub,Gym / Fitness Center,Diner,Park,Music Store
21,West Toronto,0,Mexican Restaurant,Bar,Café,Bakery,Cajun / Creole Restaurant,Italian Restaurant,Flea Market,Speakeasy,Fried Chicken Joint,Arts & Crafts Store
26,Downtown Toronto,0,Café,Bookstore,Restaurant,Japanese Restaurant,Bar,Bakery,Comfort Food Restaurant,Chinese Restaurant,Sandwich Place,Beer Store


In [90]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Park,Café,Bakery,Theater,Mexican Restaurant,Pub,Restaurant,Breakfast Spot,Gym / Fitness Center
1,Downtown Toronto,1,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Café,Fast Food Restaurant,Diner,Restaurant,Ramen Restaurant,Bookstore
2,Downtown Toronto,1,Coffee Shop,Hotel,Restaurant,Café,Italian Restaurant,Beer Bar,Breakfast Spot,Cocktail Bar,Cosmetics Shop,Gastropub
4,Downtown Toronto,1,Coffee Shop,Cocktail Bar,Farmers Market,Bakery,Beer Bar,Steakhouse,Cheese Shop,Seafood Restaurant,Café,Jazz Club
5,Downtown Toronto,1,Coffee Shop,Italian Restaurant,Café,Ice Cream Shop,Sandwich Place,Burger Joint,Salad Place,Spa,Sushi Restaurant,Japanese Restaurant
7,Downtown Toronto,1,Coffee Shop,Café,Thai Restaurant,Steakhouse,Bar,Cosmetics Shop,Asian Restaurant,Burger Joint,American Restaurant,Gym
9,Downtown Toronto,1,Coffee Shop,Hotel,Aquarium,Café,Italian Restaurant,Sporting Goods Shop,Scenic Lookout,Bakery,Brewery,Pizza Place
10,West Toronto,1,Bar,Coffee Shop,Asian Restaurant,French Restaurant,Café,Pizza Place,New American Restaurant,Men's Store,Wine Bar,Cocktail Bar
11,East Toronto,1,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Cosmetics Shop,Bookstore,Brewery,Bubble Tea Shop,Restaurant
12,Downtown Toronto,1,Coffee Shop,Café,Hotel,Italian Restaurant,Restaurant,American Restaurant,Gastropub,Gym,Bar,Deli / Bodega


In [91]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,2,Home Service,Garden,Wings Joint,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
